In [2]:
# '''Trains two recurrent neural networks based upon a story and a question.
# The resulting merged vector is then queried to answer a range of bAbI tasks.
# The results are comparable to those for an LSTM model provided in Weston et al.:
# "Towards AI-Complete Question Answering: A Set of Prerequisite Toy Tasks"
# http://arxiv.org/abs/1502.05698
# Task Number                  | FB LSTM Baseline | Keras QA
# ---                          | ---              | ---
# QA1 - Single Supporting Fact | 50               | 52.1
# QA2 - Two Supporting Facts   | 20               | 37.0
# QA3 - Three Supporting Facts | 20               | 20.5
# QA4 - Two Arg. Relations     | 61               | 62.9
# QA5 - Three Arg. Relations   | 70               | 61.9
# QA6 - yes/No Questions       | 48               | 50.7
# QA7 - Counting               | 49               | 78.9
# QA8 - Lists/Sets             | 45               | 77.2
# QA9 - Simple Negation        | 64               | 64.0
# QA10 - Indefinite Knowledge  | 44               | 47.7
# QA11 - Basic Coreference     | 72               | 74.9
# QA12 - Conjunction           | 74               | 76.4
# QA13 - Compound Coreference  | 94               | 94.4
# QA14 - Time Reasoning        | 27               | 34.8
# QA15 - Basic Deduction       | 21               | 32.4
# QA16 - Basic Induction       | 23               | 50.6
# QA17 - Positional Reasoning  | 51               | 49.1
# QA18 - Size Reasoning        | 52               | 90.8
# QA19 - Path Finding          | 8                | 9.0
# QA20 - Agent's Motivations   | 91               | 90.7
# For the resources related to the bAbI project, refer to:
# https://research.facebook.com/researchers/1543934539189348
# # Notes
# - With default word, sentence, and query vector sizes, the GRU model achieves:
#   - 52.1% test accuracy on QA1 in 20 epochs (2 seconds per epoch on CPU)
#   - 37.0% test accuracy on QA2 in 20 epochs (16 seconds per epoch on CPU)
# In comparison, the Facebook paper achieves 50% and 20% for the LSTM baseline.
# - The task does not traditionally parse the question separately. This likely
# improves accuracy and is a good example of merging two RNNs.
# - The word vector embeddings are not shared between the story and question RNNs.
# - See how the accuracy changes given 10,000 training samples (en-10k) instead
# of only 1000. 1000 was used in order to be comparable to the original paper.
# - Experiment with GRU, LSTM, and JZS1-3 as they give subtly different results.
# - The length and noise (i.e. 'useless' story components) impact the ability of
# LSTMs / GRUs to provide the correct answer. Given only the supporting facts,
# these RNNs can achieve 100% accuracy on many tasks. Memory networks and neural
# networks that use attentional processes can efficiently search through this
# noise to find the relevant statements, improving performance substantially.
# This becomes especially obvious on QA2 and QA3, both far longer than QA1.
# '''

In [3]:
from __future__ import print_function
from functools import reduce
import re
import tarfile

import numpy as np

from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers import recurrent
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences


Using TensorFlow backend.


In [4]:
def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split(r'(\W+)?', sent) if x.strip()]

In [5]:
def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format
    If only_supporting is true,
    only the sentences that support the answer are kept.
    '''
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data

In [6]:
def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file, retrieve the stories,
    and then convert the sentences into a single story.
    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data
            if not max_length or len(flatten(story)) < max_length]
    return data

In [7]:
def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    xs = []
    xqs = []
    ys = []
    for story, query, answer in data:
        x = [word_idx[w] for w in story]
        xq = [word_idx[w] for w in query]
#         print('x: ',np.shape(x))
#         print('xq: ',np.shape(xq))
        # let's not forget that index 0 is reserved
        y = np.zeros(len(word_idx) + 1)
        y[word_idx[answer]] = 1
        xs.append(x)
        xqs.append(xq)
        ys.append(y)
    return (pad_sequences(xs, maxlen=story_maxlen),
            pad_sequences(xqs, maxlen=query_maxlen), np.array(ys))

In [8]:
RNN = recurrent.LSTM
EMBED_HIDDEN_SIZE = 50
SENT_HIDDEN_SIZE = 100
QUERY_HIDDEN_SIZE = 100
BATCH_SIZE = 32
EPOCHS = 100
print('RNN / Embed / Sent / Query = {}, {}, {}, {}'.format(RNN,
                                                           EMBED_HIDDEN_SIZE,
                                                           SENT_HIDDEN_SIZE,
                                                           QUERY_HIDDEN_SIZE))

try:
    path = get_file('babi-tasks-v1-2.tar.gz',
                    origin='https://s3.amazonaws.com/text-datasets/'
                           'babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2'
          '.tar.gz\n'
          '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise


RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100


In [9]:
# Default QA1 with 1000 samples
# challenge = 'tasks_1-20_v1-2/en/qa1_single-supporting-fact_{}.txt'
# QA1 with 10,000 samples
# challenge = 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt'
# QA2 with 1000 samples
challenge = 'tasks_1-20_v1-2/en/qa2_two-supporting-facts_{}.txt'
# QA2 with 10,000 samples
# challenge = 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt'
with tarfile.open(path) as tar:
    train = get_stories(tar.extractfile(challenge.format('train')))
    test = get_stories(tar.extractfile(challenge.format('test')))

vocab = set()
for story, q, answer in train + test:
    vocab |= set(story + q + [answer])
vocab = sorted(vocab)

# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
story_maxlen = max(map(len, (x for x, _, _ in train + test)))
query_maxlen = max(map(len, (x for _, x, _ in train + test)))

x, xq, y = vectorize_stories(train, word_idx, story_maxlen, query_maxlen)
tx, txq, ty = vectorize_stories(test, word_idx, story_maxlen, query_maxlen)

# print('vocab = {}'.format(vocab))
# print('x.shape = {}'.format(x.shape))
# print('xq.shape = {}'.format(xq.shape))
# print('y.shape = {}'.format(y.shape))
# print('story_maxlen, query_maxlen = {}, {}'.format(story_maxlen, query_maxlen))

/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [10]:
print('story_maxlen: ',story_maxlen)

story_maxlen:  552


In [13]:
print('story shape: ',np.shape(x[0]))

story shape:  (552,)


In [11]:
print('Build model...')

sentence = layers.Input(shape=(story_maxlen,), dtype='int32')
encoded_sentence = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(sentence)
encoded_sentence = RNN(SENT_HIDDEN_SIZE)(encoded_sentence)

question = layers.Input(shape=(query_maxlen,), dtype='int32')
encoded_question = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(question)
encoded_question = RNN(QUERY_HIDDEN_SIZE)(encoded_question)

merged = layers.concatenate([encoded_sentence, encoded_question])
preds = layers.Dense(vocab_size, activation='softmax')(merged)

model = Model([sentence, question], preds)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Build model...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 552)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 5)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 552, 50)      1800        input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 5, 50)        1800        input_2[0][0]                    
______________________________________________________________________________________________

In [ ]:
# print('Training')
# model.fit([x, xq], y,
#           batch_size=BATCH_SIZE,
#           epochs=EPOCHS,
#           validation_split=0.05)

# print('Evaluation')
# loss, acc = model.evaluate([tx, txq], ty,
#                            batch_size=BATCH_SIZE)
# print('Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))